<a href="https://colab.research.google.com/github/shritigupta/EIP/blob/master/assign5/2nd_PersonAttrubutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2
from keras import backend as K



In [23]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [24]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
pixel_level= True
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [0]:
augment = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=20,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        brightness_range=(0.2,2.5),
        # set range for random shear
        shear_range=0.2,
        # set range for random zoom
        zoom_range=0.5,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=1.0/255,
        # set function that will be applied on each input
        #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=pixel_level),
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.)

In [0]:
import keras
import numpy as np
import scipy

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=128, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]

        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
      


        if self.augmentation is not None:
          image = self.augmentation.flow(image,shuffle=False).next()
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [28]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.25)
train_df.shape, val_df.shape

((10858, 28), (2715, 28))

In [29]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
7163,resized/7164.jpg,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0
11284,resized/11286.jpg,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1
6872,resized/6873.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0
12364,resized/12366.jpg,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0
7409,resized/7410.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, shuffle=True, augmentation=augment)
valid_gen = PersonDataGenerator(val_df, batch_size=32, shuffle=False, 
                                augmentation=ImageDataGenerator(rescale=1.0/255,rotation_range=20,))

In [31]:
# get number of output units from data

images, targets = next(iter(train_gen))
print(images.shape)
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

(32, 224, 224, 3)


{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 4:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v1(input_shape, depth, num_classes=27):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs,
                     activation='relu')
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = BatchNormalization()(x)
    Conv2D(num_filters,
                  kernel_size=3,
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4),
                  activation="relu")(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation='relu')
            #y = MaxPooling2D(pool_size=(2,2))(y)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation='relu',
                                 batch_normalization=False)
                #x = MaxPooling2D(pool_size=(2,2))(x)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)

        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    #x = GlobalAveragePooling2D()(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='sigmoid',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [72]:
'''
backbone = ResNet50(
    weights=None, 
    include_top=True, 
    #input_tensor=Input(shape=(224, 224, 3)),
    classes=8,
    )
'''

n = 5
depth = n * 6 + 2

#input_shape = (224, 224, 3)
backbone = resnet_v1(input_shape=(224,224,3), depth=depth, num_classes=27)

neck = backbone.output
#neck = Flatten(name="flatten")(neck) 
#neck = GlobalAveragePooling2D()(neck)
#neck = BatchNormalization()(neck)
neck = Dropout(0.2)(neck)
neck = Dense(27, activation="relu",kernel_regularizer=l2(1e-4))(neck)


def build_tower(in_layer):
    neck = Dropout(0.25)(in_layer)
    neck = Dense(27, activation="relu",kernel_regularizer=l2(1e-4))(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(27, activation="relu",kernel_regularizer=l2(1e-4))(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="sigmoid", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

model.summary()

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_209 (Conv2D)             (None, 224, 224, 16) 448         input_14[0][0]                   
__________________________________________________________________________________________________
batch_normalization_195 (BatchN (None, 224, 224, 16) 64          conv2d_209[0][0]                 
__________________________________________________________________________________________________
activation_293 (Activation)     (None, 224, 224, 16) 0           batch_normalization_195[0][0]    
___________________________________________________________________________________________

In [0]:
# freeze backbone
for layer in backbone.layers[0:(len(backbone.layers)-1)]:
	layer.trainable = False

In [0]:
 losses = {
 	"gender_output": "binary_crossentropy",
 	"image_quality_output": "categorical_crossentropy",
 	"age_output": "categorical_crossentropy",
 	"weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "emotion_output":"categorical_crossentropy",

 }
 '''
 loss_weights = {"gender_output": 0.5, "image_quality_output": 1.0, "age_output": 0.8,
                 "weight_output": 1.0,
                 "bag_output": 1.0,
                 "pose_output": 1.0,
                 "footwear_output": 1.0,
                 "emotion_output":1.0,
                 }
'''
epochs=40
learning_rate = 0.001
opt = SGD(lr=0.001, momentum=0.9,decay=1e-6, nesterov=True,clipnorm=1.)
#opt = SGD(lr=0.001, momentum=0.6,clipvalue=1.)
model.compile(
    optimizer=opt,
    loss='binary_crossentropy', 
    #loss=losses,
    #loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)

In [0]:
from collections import Counter
import os

my_classes = {"gender_output", 
                 "image_quality_output",
                 "age_output",
                 "weight_output",
                 "bag_output",
                 "pose_output",
                 "footwear_output",
                 "emotion_output",
                 }
#itemCt = 
#maxCt = float(max(itemCt.values()))
#cw = {clsID : maxCt/numImg for clsID, numImg in itemCt.items()}

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.

checkpoint = ModelCheckpoint(filepath=filepath,
                             verbose=1,
                             save_best_only=True)
                             



lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)



callbacks = [checkpoint]


In [95]:
import keras.backend as K
from keras.models import load_model


model = load_model('/content/saved_models/model.011.h5')
# update the learning rate
print("[INFO] old learning rate: {}".format(
		K.get_value(model.optimizer.lr)))
K.set_value(model.optimizer.lr, 0.00001)
print("[INFO] new learning rate: {}".format(
		K.get_value(model.optimizer.lr)))

print(len(train_gen))

model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=50,
    verbose=1,
    #class_weight=cw,
    #steps_per_epoch=180, # 128
    callbacks=callbacks,
)

[INFO] old learning rate: 9.999999747378752e-05
[INFO] new learning rate: 9.999999747378752e-06
339
Epoch 1/50
339/339 [==============================] - 98s 289ms/step - loss: 4.5083 - gender_output_loss: 0.6864 - image_quality_output_loss: 0.5766 - age_output_loss: 0.4578 - weight_output_loss: 0.4229 - bag_output_loss: 0.5509 - footwear_output_loss: 0.6107 - pose_output_loss: 0.5459 - emotion_output_loss: 0.3874 - gender_output_acc: 0.5596 - image_quality_output_acc: 0.7040 - age_output_acc: 0.8000 - weight_output_acc: 0.8178 - bag_output_acc: 0.7080 - footwear_output_acc: 0.6664 - pose_output_acc: 0.7453 - emotion_output_acc: 0.8558 - val_loss: 4.4977 - val_gender_output_loss: 0.6836 - val_image_quality_output_loss: 0.5816 - val_age_output_loss: 0.4567 - val_weight_output_loss: 0.4210 - val_bag_output_loss: 0.5466 - val_footwear_output_loss: 0.6041 - val_pose_output_loss: 0.5485 - val_emotion_output_loss: 0.3860 - val_gender_output_acc: 0.5740 - val_image_quality_output_acc: 0.6957 

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.192831). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


339/339 [==============================] - 87s 255ms/step - loss: 4.5094 - gender_output_loss: 0.6868 - image_quality_output_loss: 0.5770 - age_output_loss: 0.4578 - weight_output_loss: 0.4229 - bag_output_loss: 0.5508 - footwear_output_loss: 0.6107 - pose_output_loss: 0.5463 - emotion_output_loss: 0.3875 - gender_output_acc: 0.5596 - image_quality_output_acc: 0.7042 - age_output_acc: 0.8000 - weight_output_acc: 0.8178 - bag_output_acc: 0.7080 - footwear_output_acc: 0.6669 - pose_output_acc: 0.7455 - emotion_output_acc: 0.8559 - val_loss: 4.4978 - val_gender_output_loss: 0.6838 - val_image_quality_output_loss: 0.5818 - val_age_output_loss: 0.4567 - val_weight_output_loss: 0.4210 - val_bag_output_loss: 0.5466 - val_footwear_output_loss: 0.6039 - val_pose_output_loss: 0.5480 - val_emotion_output_loss: 0.3864 - val_gender_output_acc: 0.5740 - val_image_quality_output_acc: 0.6957 - val_age_output_acc: 0.8000 - val_weight_output_acc: 0.8179 - val_bag_output_acc: 0.7140 - val_footwear_output

In [0]:
model